In [1]:
import string
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import random
tqdm.pandas()

In [2]:
file_path = 'LK_modified.xlsx'
all_sheets = pd.read_excel(file_path, sheet_name=None)

dfs = {sheet_name: pd.DataFrame(sheet_data) for sheet_name, sheet_data in all_sheets.items()}


qa_df = dfs[list(dfs.keys())[0]]
synthetic = dfs[list(dfs.keys())[1]]
popular_phrases = dfs[list(dfs.keys())[2]]
glossary = dfs[list(dfs.keys())[3]]

In [3]:
qa_df

,id,question,content,category,source
0,0,"Я сменил автомобить, на учет еще не поставил, ...",Для внесения данных по личному автомобилю обра...,Автомобиль,оригинал
1,1,Не отображается автомобиль в личном кабинете.,Для внесения данных по личному автомобилю обра...,Автомобиль,оригинал
2,2,добавить автомобиль,Для внесения данных по личному автомобилю обра...,Автомобиль,оригинал
3,3,хочу внести данные об автомобиле,Для внесения данных по личному автомобилю обра...,Автомобиль,оригинал
4,4,Как внести данные об автомобиле?,Для внесения данных по личному автомобилю обра...,Автомобиль,оригинал
...,...,...,...,...,...
5784,5784,как сделать продление срочного трудового договора,Для продления срочного трудового договора созд...,График работы,синтетика
5785,5785,как продлить договор,Для продления срочного трудового договора созд...,График работы,синтетика
5786,5786,как перевести сотрудника на бессрочный ТД,Для продления срочного трудового договора созд...,График работы,синтетика
5787,5787,как продлить трудовой договор?,Для продления срочного трудового договора созд...,График работы,синтетика


In [4]:
def encode_column(df, column_name):
  id_map = {}
  for x in df[column_name]:
    id_map[x] = id_map.get(x, len(id_map))
  df[f'{column_name}_id'] = df[column_name].map(id_map)
  return id_map

In [5]:
id_map_content = encode_column(qa_df, 'content')
id_map_categories = encode_column(qa_df, 'category')

In [6]:
qa_df['content_count'] = qa_df['content_id'].map(qa_df.groupby('content_id').agg('size'))

In [7]:
def add_or_remove_punctuation(text):
    """Добавление или удаление знаков препинания."""
    # Возможные варианты добавления знаков препинания
    punctuations = [',', '.', '!', '?']
    words = text.split()

    # Добавляем или удаляем знаки препинания
    if random.random() < 0.5:
        # Добавить знак препинания
        position = random.randint(0, len(words) - 1)
        punct = random.choice(punctuations)
        words[position] = words[position] + punct
    else:
        # Удалить знак препинания, если он есть
        text = text.translate(str.maketrans('', '', string.punctuation))
        words = text.split()

    return ' '.join(words)

In [8]:
def introduce_typo(text):
    """Создание опечаток в тексте."""
    if not text:
        return text

    words = text.split()
    index = random.randint(0, len(words) - 1)
    word = words[index]

    # Опечатки: замена, пропуск или дублирование символов
    typo_type = random.choice(['swap', 'remove', 'duplicate'])

    if typo_type == 'swap' and len(word) > 1:
        # Меняем местами соседние буквы
        pos = random.randint(0, len(word) - 2)
        word = list(word)
        word[pos], word[pos + 1] = word[pos + 1], word[pos]
        words[index] = ''.join(word)

    elif typo_type == 'remove' and len(word) > 1:
        # Удаляем случайную букву
        pos = random.randint(0, len(word) - 1)
        words[index] = word[:pos] + word[pos + 1:]

    elif typo_type == 'duplicate':
        # Дублируем случайную букву
        pos = random.randint(0, len(word) - 1)
        words[index] = word[:pos] + word[pos] + word[pos:]

    return ' '.join(words)

In [9]:
def shuffle_words(text):
    """Перестановка порядка слов."""
    words = text.split()
    if len(words) > 1:
        random.shuffle(words)
    return ' '.join(words)

In [10]:
AUG_NUM = 30

def balance_dataset(qa_df):
    # Шаг 1: Найти самый частовстречаемый ответ
    max_count = qa_df['content_id'].value_counts().max()

    # Шаг 2: Сбалансировать выборку ответов
    augmented_data = []
    for content_id, group in qa_df.groupby('content_id'):
        count = len(group)
        augmented_data.extend(group.to_dict('records'))  # Добавляем все исходные строки

        # Если ответ встречается реже, чем самый частовстречаемый, создаем аугментированные копии вопросов
        for _ in range(min(AUG_NUM, max_count - count)):
            row = group.sample(1).iloc[0].to_dict()  # Случайный вопрос из группы
            question = row['question']

            # Применяем несколько аугментаций последовательно
            augmented_question = add_or_remove_punctuation(question)
            augmented_question = introduce_typo(augmented_question)
            augmented_question = shuffle_words(augmented_question)

            # Сохраняем аугментированный вопрос с исходным ответом
            new_row = row.copy()
            new_row['question'] = augmented_question
            augmented_data.append(new_row)

    # Шаг 3: Создать новый сбалансированный датафрейм
    balanced_qa_df = pd.DataFrame(augmented_data)
    return balanced_qa_df

In [11]:
def process_splitted_data(df, do_aug=True):
    augmented_df = df.copy()
    if do_aug:
        augmented_df = balance_dataset(df)

    augmented_df['question'] = augmented_df['category'] + " " + augmented_df['question']
    augmented_df['question'] = augmented_df['question'].str.lower()

    return augmented_df

In [12]:
def stratified_train_test_split(df, content_column, source_column, test_size=0.2, random_state=42, with_synthetic=True):
    """
    Разделение данных на обучающую и тестовую выборки с учетом следующих шагов:
    1) Если with_synthetic=True:
        - Оригинальные данные из source_column.
        - Наблюдения, которые встречаются 1 раз по content_column, откладываются отдельно.
        - Стратифицированное разделение на train/test.
        - Единичные наблюдения добавляются в тест.
        - Синтетические данные добавляются в train.
    2) Если with_synthetic=False:
        - Фильтрация данных от частотных наблюдений (>= 3).
        - Стратифицированное базовое разделение на train/test.
    """

    if with_synthetic:
        # Шаги для работы с синтетикой
        original_data = df[(df[source_column] == 'оригинал') | (df[source_column] == 'популярные')]
        content_counts = original_data[content_column].value_counts()
        single_content = content_counts[content_counts == 1].index
        single_data = original_data[original_data[content_column].isin(single_content)]

        remaining_data = original_data[~original_data[content_column].isin(single_content)]

        train, test = train_test_split(remaining_data, test_size=test_size,
                                                 stratify=remaining_data[content_column],
                                                 random_state=random_state)

        test = pd.concat([test, single_data])

        synthetic_data = df[df[source_column] == 'синтетика']
        train = pd.concat([train, synthetic_data])

    else:
        # Шаги для работы без синтетики
        original_data = df[(df[source_column] == 'оригинал') | ((df[source_column] == 'популярные'))].copy()
        original_data['content_count'] = original_data['content_id'].map(original_data['content_id'].value_counts())
        filtered_df = original_data[original_data['content_count'] >= 3]

        train, test = train_test_split(filtered_df, test_size=test_size,
                                           stratify=filtered_df[content_column],
                                           random_state=random_state)


    return train, test


In [13]:
train, test = stratified_train_test_split(qa_df, content_column='content', source_column='source', test_size=0.5, random_state=42, with_synthetic=True)

In [14]:
train_processed = process_splitted_data(train, do_aug=False)
test_processed = process_splitted_data(test, do_aug=False)

In [15]:
train_processed[['id', 'question', 'content', 'category', 'source']].to_csv('train_synthetic_no_aug.csv', index=False)
test_processed[['id', 'question', 'content', 'category', 'source']].to_csv('test_synthetic_no_aug.csv', index=False)

In [16]:
train_processed = process_splitted_data(train, do_aug=True)
test_processed = process_splitted_data(test, do_aug=False)

In [17]:
train_processed[['id', 'question', 'content', 'category', 'source']].to_csv('train_synthetic_aug.csv', index=False)
test_processed[['id', 'question', 'content', 'category', 'source']].to_csv('test_synthetic_aug.csv', index=False)

In [18]:
train_processed[['id', 'question', 'content', 'category', 'source']]

,id,question,content,category,source
0,2,автомобиль добавить автомобиль,Для внесения данных по личному автомобилю обра...,Автомобиль,оригинал
1,7,"автомобиль при смене автомобиля, который испол...",Для внесения данных по личному автомобилю обра...,Автомобиль,оригинал
2,4,автомобиль как внести данные об автомобиле?,Для внесения данных по личному автомобилю обра...,Автомобиль,оригинал
3,5,автомобиль мне нужно внести данные об автомобиле,Для внесения данных по личному автомобилю обра...,Автомобиль,оригинал
4,5,автомобиль данные. нужно вести об мне автомобиле,Для внесения данных по личному автомобилю обра...,Автомобиль,оригинал
...,...,...,...,...,...
11404,1682,поддержка бот опмогает не,Чат-бот находится в стадии пилотирования и обу...,Поддержка,популярные
11405,1686,поддержка наа портал вернуться старый,Чат-бот находится в стадии пилотирования и обу...,Поддержка,популярные
11406,1682,"поддержка не, помогает бо",Чат-бот находится в стадии пилотирования и обу...,Поддержка,популярные
11407,1686,поддержка портал старый нна вернуться,Чат-бот находится в стадии пилотирования и обу...,Поддержка,популярные
